In [1]:
!pip install cryptofeed redis nest_asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.6/610.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 18.3 MB/s eta 0:00:00
  Created wheel for cryptofeed: filename=cryptofeed-2.4.1-cp311-cp311-linux_x86_64.whl size=1601205 sha256=6d57bab6d87c8e4f36bc4ef7f4c35e22a63628aecf07c7fee095e316cb647397
  Stored in directory: /root/.cache/pip/wheels/0a/30/84/dcc1003b05dd7598f9808f2a6e12faeaf4230781aba5c1a205
Successfully built cryptofeed

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import time
from decimal import Decimal
from cryptofeed import FeedHandler
from cryptofeed.defines import TRADES
from cryptofeed.exchanges import Bitfinex
import redis

# Redis Cloud connection details
redis_client = redis.Redis(
    host='redis-19129.c262.us-east-1-3.ec2.redns.redis-cloud.com',
    port=19129,
    decode_responses=True,
    username='satvik',
    password='Stankar@123',
)

trade_count = 0

async def trade_callback(t, receipt_timestamp):
    global trade_count
    trade_count += 1

    price = float(t.price) if isinstance(t.price, Decimal) else t.price
    amount = float(t.amount) if isinstance(t.amount, Decimal) else t.amount

    redis_key = f"bitfinex:{int(t.timestamp)}:{trade_count}"
    trade_doc = {
        "exchange": t.exchange,
        "symbol": t.symbol,
        "side": t.side,
        "price": price,
        "amount": amount,
        "exchange_ts": t.timestamp,
        "received_ts": receipt_timestamp,
        "latency_secs": receipt_timestamp - t.timestamp
    }

    try:
        result = redis_client.hset(redis_key, mapping=trade_doc)
        redis_client.expire(redis_key, 60 * 60 * 24)
        print(f"✅ Trade #{trade_count} saved: {bool(result)}\n   exchange: {t.exchange}\n   symbol: {t.symbol}\n   side: {t.side}\n   price: {price}\n   amount: {amount}\n   exchange_ts: {t.timestamp}\n   received_ts: {receipt_timestamp}\n   latency_secs: {receipt_timestamp - t.timestamp:.6f}")
    except Exception as e:
        print(f"❌ Redis error: {e}")

    if trade_count >= 10:
        print("✅ Collected 10 trades – stopping…")
        fh.stop()

fh = FeedHandler()
fh.add_feed(Bitfinex(
    symbols=["BTC-USDT"],
    channels=[TRADES],
    callbacks={TRADES: trade_callback}
))

print("🚀 Starting collection of 10 Bitfinex BTC-USDT trades...")

try:
    fh.run()
except KeyboardInterrupt:
    print("❌ Stopped by user.")
except Exception as e:
    print(f"❌ FeedHandler error: {e}")

print("🕒 Sleeping for 10 minutes...")
time.sleep(20)
print("✅ Script execution completed.")

🚀 Starting collection of 10 Bitfinex BTC-USDT trades...
✅ Trade #1 saved: True
   exchange: BITFINEX
   symbol: BTC-USDT
   side: sell
   price: 88389.0
   amount: 0.00604091
   exchange_ts: 1745301881.201
   received_ts: 1745301909.4350617
   latency_secs: 28.234062
✅ Trade #2 saved: True
   exchange: BITFINEX
   symbol: BTC-USDT
   side: sell
   price: 88396.0
   amount: 0.01579885
   exchange_ts: 1745301881.2
   received_ts: 1745301909.4350617
   latency_secs: 28.235062
✅ Trade #3 saved: True
   exchange: BITFINEX
   symbol: BTC-USDT
   side: sell
   price: 88393.0
   amount: 0.0002
   exchange_ts: 1745301881.2
   received_ts: 1745301909.4350617
   latency_secs: 28.235062
✅ Trade #4 saved: True
   exchange: BITFINEX
   symbol: BTC-USDT
   side: sell
   price: 88396.0
   amount: 0.0002
   exchange_ts: 1745301881.199
   received_ts: 1745301909.4350617
   latency_secs: 28.236062
✅ Trade #5 saved: True
   exchange: BITFINEX
   symbol: BTC-USDT
   side: buy
   price: 88397.0
   amount: 0